# **Emojis para Análise de Sentimentos**

## Download e carregamento dos dados
A Unicode disponibiliza os dados dos emojis mais usados tanto em 2019 quanto em 2021.  
O link está na seguinte notícia da página oficial: https://home.unicode.org/emoji/emoji-frequency/  
Caso prefira o link direto para a pasta do drive: [emoji - Google Drive](https://drive.google.com/drive/folders/18u0N7OOjbP-V2YolddTPaMojhNFavmmF?usp=drive_link)

In [1]:
import pandas as pd
import requests
from io import StringIO

# URL do arquivo do Google Drive (o link abaixo corresponde ao arquivo "2021_ranked.tsv")
url = "https://drive.google.com/uc?export=download&id=1ZdI3WjETaxp-O92bAhjpq4NXL3rDdhx_"

response = requests.get(url)
content = response.content.decode('utf-8')

df_emojis = pd.read_csv(StringIO(content), sep='\t')

df_emojis.head()

,Hex,Rank,Emoji,Year,Category,Subcategory,Name
0,\x{1F602},1,😂,2010,Smileys & Emotion,face-smiling,face with tears of joy
1,\x{2764 FE0F},2,❤️,2010,Smileys & Emotion,emotion,red heart
2,\x{1F923},3,🤣,2016,Smileys & Emotion,face-smiling,rolling on the floor laughing
3,\x{1F44D},4,👍,2010,People & Body,hand-fingers-closed,thumbs up
4,\x{1F62D},5,😭,2010,Smileys & Emotion,face-concerned,loudly crying face


## Filtrando o DataFrame
O DataFrame emojis foi filtrado e organizado através dos seguintes processos:
* Aplicação de filtro para seleção dos emojis que a subcategoria era relacionada à faces (rostos);
* Exclusão das colunas "Rank" e "Year", relacionadas à posição do emoji no ranking e ao ano de lançamento do emoji, respectivamente;
* Reordenação das colunas.

In [2]:
emojis = df_emojis.loc[df_emojis['Subcategory'].str.contains("face")]
emojis = emojis.drop(columns=['Rank', 'Year'])
emojis = emojis.reindex(columns=['Emoji', 'Hex', 'Category','Subcategory', 'Name'])
emojis.head()

,Emoji,Hex,Category,Subcategory,Name
0,😂,\x{1F602},Smileys & Emotion,face-smiling,face with tears of joy
2,🤣,\x{1F923},Smileys & Emotion,face-smiling,rolling on the floor laughing
4,😭,\x{1F62D},Smileys & Emotion,face-concerned,loudly crying face
6,😘,\x{1F618},Smileys & Emotion,face-affection,face blowing a kiss
7,🥰,\x{1F970},Smileys & Emotion,face-affection,smiling face with hearts


### Salvando os dados dos emojis
Essa é uma etapa opcional, mas caso seja do seu interesse salvar apenas os dados já filtrados dos emojis, é só executar a célula abaixo.

In [3]:
emojis.to_csv("emojis_face.csv", index=False)

## Adicionando a Análise de Sentimentos
A análise de sentimentos foi realizada utilizando a biblioteca [pysentimiento](https://github.com/pysentimiento/pysentimiento)¹

---
¹<small>Artigo original: [pysentimiento: A Python Toolkit for Opinion Mining and Social NLP tasks](https://arxiv.org/abs/2106.09462) </small>

### Instalando as bibliotecas necessárias

In [4]:
!pip install --upgrade --quiet transformers
!pip install --upgrade --quiet pyarrow
!pip install --quiet pysentimiento

### Criando os analisadores
Nesse projeto foram utilizados:
* **Analisador de sentimentos para PT-BR**: analisa um texto ou emoji e retorna a polaridade dele apontando se é _positivo, negativo ou neutro_;
* **Analisador de emoções para EN**: retorna uma emoção relacionada ao texto analisado, podendo ser _joy, surprise, disgust, sadness, fear, anger ou others_.

In [5]:
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="pt")
emotion_analyzer = create_analyzer(task="emotion", lang="en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/799k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

### Realizando as análises
Cada análise foi realizada através de funções lambda, retornando cada um dos objetos de interesse para a coluna.
> #### Detalhes das funções  
> * `sentiment_analyzer.predict(x).output`: retorna a polaridade do emoji;  
> * `sentiment_analyzer.predict(x).probas['NEG']`: retorna a probabilidade do emoji ser negativo;
> * `sentiment_analyzer.predict(x).probas['NEU']`: retorna a probabilidade do emoji ser neutro;
> * `sentiment_analyzer.predict(x).probas['POS']`: retorna a probabilidade do emoji ser positivo;
> * `emotion_analyzer.predict(x).output`: retorna a emoção com a maior probabilidade de relação com o emoji.

In [6]:
emojis['emoji_polaridade_pysent'] = emojis['Emoji'].apply(lambda x: sentiment_analyzer.predict(x).output)
emojis['emoji_neg_pysent'] = emojis['Emoji'].apply(lambda x: sentiment_analyzer.predict(x).probas['NEG'])
emojis['emoji_neu_pysent'] = emojis['Emoji'].apply(lambda x: sentiment_analyzer.predict(x).probas['NEU'])
emojis['emoji_pos_pysent'] = emojis['Emoji'].apply(lambda x: sentiment_analyzer.predict(x).probas['POS'])
emojis['emoji_emotion_pysent'] = emojis['Emoji'].apply(lambda x: emotion_analyzer.predict(x).output)

emojis.head()

,Emoji,Hex,Category,Subcategory,Name,emoji_polaridade_pysent,emoji_neg_pysent,emoji_neu_pysent,emoji_pos_pysent,emoji_emotion_pysent
0,😂,\x{1F602},Smileys & Emotion,face-smiling,face with tears of joy,POS,0.072761,0.270592,0.656647,joy
2,🤣,\x{1F923},Smileys & Emotion,face-smiling,rolling on the floor laughing,POS,0.049749,0.131189,0.819062,others
4,😭,\x{1F62D},Smileys & Emotion,face-concerned,loudly crying face,NEG,0.952707,0.025290,0.022004,sadness
6,😘,\x{1F618},Smileys & Emotion,face-affection,face blowing a kiss,POS,0.046458,0.397514,0.556029,others
7,🥰,\x{1F970},Smileys & Emotion,face-affection,smiling face with hearts,POS,0.001380,0.016386,0.982234,joy


### Salvando planilha com as análises

In [7]:
emojis.to_csv("emojis-sentimentos_emocoes.csv", index=False)